# R example notebook for NoobQuant conda environments

Tested with *dev2018* and *dev2021*.

## Show paths of R.exe and import paths

In [ ]:
print(file.path(R.home("bin"), "R"))
print(.libPaths())

## Test imports

In [ ]:
library(tidyverse)
library(ggplot2)
library(RJSDMX)
library(datasets)

## Test tidyverse (tidyr, stringr, lubridate, ggplot2) and base packages

In [ ]:
# Get data
df = txhousing

In [ ]:
# Calculate the distinct number of observations per city and the distinct number of min/max dates.
# This is to confirm we have a panel dataset wihthout gaps.
df %>% group_by(city) %>%
    summarise(
        count=n(),
        mindate = min(date),
        maxdate = max(date),
    ) %>%
    ungroup() %>%
    summarise(
        count_ok = case_when(n_distinct(count) == 1 ~ "OK", TRUE ~ "NO"),
        mindate_ok = case_when(n_distinct(mindate) == 1 ~ "OK", TRUE ~ "NO"),
        maxdate_ok = case_when(n_distinct(maxdate) == 1 ~ "OK", TRUE ~ "NO")
    )

In [ ]:
# Create date variables
df = df %>%
    mutate(
        firstday = as.Date(paste(year, month, "01", sep="-")),
        yrmonth = format(firstday, "%Y-%m")
    )

# Check dates are correctly calculated
df %>% filter(year == 2000) %>%
    distinct(year, month, firstday, yrmonth)

In [ ]:
# Calculate lagged volume variable in two ways (dplyr and lubridate), check that they match
df = df %>%
    group_by(city) %>%
    mutate(volume_lag = dplyr::lag(volume, order_by=firstday))

df_volume_lag = df %>%
    select(city, firstday, volume) %>%
    mutate(firstday_lead = firstday + months(1)) %>%
    select(city, firstday_lead, volume)
colnames(df_volume_lag) = c("city", "firstday", "volume_lag2")

df = df %>%
    left_join(df_volume_lag)
head(df %>% select(city, firstday, volume, volume_lag, volume_lag2))

In [ ]:
# Divide montly observatios in terciles based on on the number of listings
df_terciles = df %>%
    group_by(firstday) %>%
    summarise(
        P33 = quantile(listings, probs=1/3, na.rm=TRUE),
        P67 = quantile(listings, probs=2/3, na.rm=TRUE)
    )
df = df %>%
    left_join(df_terciles) %>%
    mutate(
        listing_tercile = case_when(
            listings > P67 ~ "TOP",
            listings > P33 & listings <= P67 ~ "MID",
            listings <= P33 ~ "BOTTOM",
            TRUE ~ NA_character_
        )
    ) %>%
    select(-c(P33, P67))

In [ ]:
# Test ggplot2; calculate and plot monthly average sales price by listing terciles (dropping NAs)
df_terciles_sum = df %>%
    filter(!is.na(listing_tercile)) %>%
    group_by(listing_tercile, firstday) %>%
    summarise(meanprice = mean(median, na.rm=TRUE))

df_terciles_sum %>%
    ggplot(
        aes(
            x=firstday,
            y=meanprice,
            group=listing_tercile,
            color=listing_tercile
        )
    ) + geom_line() +
    scale_x_date(date_breaks="year", date_labels="%Y") + 
    xlab("")

## Test RJSDMX

Data fetch from different public APIs, here from OECD.

In [ ]:
options(warn=-1)
library(RJSDMX)
ticker = "EO.FIN.GDPV.Q"
mylist = getTimeSeries("OECD", ticker)

df = sdmxdf(mylist, meta=F)
df[["TIME_PERIOD"]] = as.Date(as.yearqtr(as.character(df[["TIME_PERIOD"]]), format = "%Y Q%q"))

plot(df[["TIME_PERIOD"]],df[['OBS_VALUE']])